In [43]:
def count_images(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".png"):
                count += 1
    return count

# Count images in train and validation sets
train_hateful_count = count_images(train_hateful_path)
train_not_hateful_count = count_images(train_not_hateful_path)
validation_hateful_count = count_images(validation_hateful_path)
validation_not_hateful_count = count_images(validation_not_hateful_path)

# Print the results
print(f"Number of hateful images in the train set: {train_hateful_count}")
print(f"Number of not hateful images in the train set: {train_not_hateful_count}")
print(f"Number of hateful images in the validation set: {validation_hateful_count}")
print(f"Number of not hateful images in the validation set: {validation_not_hateful_count}")


Number of hateful images in the train set: 2400
Number of not hateful images in the train set: 2400
Number of hateful images in the validation set: 600
Number of not hateful images in the validation set: 600


In [41]:
!pip install transformers

  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 406.8 kB/s eta 0:00:00
Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ---------------------------------------  266.2/269.0 kB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 269.0/269.0 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.9/2.2 MB 18.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.8/2.2 MB 19.3 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 17.7 MB/s eta 0:00:00


In [1]:
from transformers import AutoImageProcessor, AutoModel
from transformers import ConditionalDetrImageProcessor, ConditionalDetrForObjectDetection
from PIL import Image
import os
import torch
from collections import Counter

In [2]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/conditional-detr-resnet-50")
model = AutoModel.from_pretrained("microsoft/conditional-detr-resnet-50")

In [3]:
base_folder = r'C:\Users\rohit\Desktop\hate\hateful_memes\img\validation'
categories = ["hateful", "not_hateful"]

In [4]:
object_counter = Counter()

In [5]:
# Get a list of all image files in the base folder
image_files = [f for f in os.listdir(base_folder) if f.endswith('.jpg') or f.endswith('.png')]

# Loop through the images and perform inference
for image_file in image_files:
    image_path = os.path.join(base_folder, image_file)
    image = Image.open(image_path)

    # Prepare the inputs
    inputs = processor(images=image, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)

    # Print the outputs to understand their structure
    print(f"Outputs for image {image_file}:")
    print(outputs)

In [6]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
from PIL import Image
import torch

# Load the image processor and model
image_processor = AutoImageProcessor.from_pretrained("microsoft/conditional-detr-resnet-50")
model = AutoModelForObjectDetection.from_pretrained("microsoft/conditional-detr-resnet-50")

# Load a sample image (replace with your own image path)
image_path = r"C:\Users\rohit\Desktop\hate\hateful_memes\img\01236.png"
image = Image.open(image_path).convert("RGB")

# Prepare the image for the model
inputs = image_processor(images=[image], return_tensors="pt")

# Perform inference
outputs = model(**inputs)

# Process the outputs - extract detected objects dynamically
# Get the logits for the labels
logits = outputs.logits
# Get the predicted labels
predicted_labels = torch.argmax(logits, -1)

# Print detected objects
for label in predicted_labels[0]:
    detected_object = model.config.id2label[label.item()]
    print(f"Detected {detected_object}")

Detected person
Detected person
Detected person
Detected sheep
Detected person
Detected person
Detected backpack
Detected sheep
Detected person
Detected tie
Detected person
Detected tie
Detected sheep
Detected person
Detected sheep
Detected person
Detected person
Detected sheep
Detected handbag
Detected person
Detected sheep
Detected person
Detected person
Detected sheep
Detected person
Detected handbag
Detected tie
Detected person
Detected person
Detected sheep
Detected person
Detected person
Detected person
Detected tie
Detected person
Detected person
Detected person
Detected sheep
Detected sheep
Detected sheep
Detected person
Detected sheep
Detected person
Detected person
Detected person
Detected person
Detected person
Detected sheep
Detected tie
Detected person
Detected person
Detected person
Detected person
Detected person
Detected sheep
Detected sheep
Detected sheep
Detected person
Detected sheep
Detected sheep
Detected sheep
Detected sheep
Detected person
Detected person
Detecte

In [7]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
from PIL import Image
import torch

# Load the image processor and model
image_processor = AutoImageProcessor.from_pretrained("microsoft/conditional-detr-resnet-50")
model = AutoModelForObjectDetection.from_pretrained("microsoft/conditional-detr-resnet-50")

# Load a sample image (replace with your own image path)
image_path = r"C:\Users\rohit\Desktop\hate\hateful_memes\img\01236.png"
image = Image.open(image_path).convert("RGB")

# Prepare the image for the model
inputs = image_processor(images=[image], return_tensors="pt")

# Perform inference
outputs = model(**inputs)

# Extract logits, bounding boxes, and labels
logits = outputs.logits
bboxes = outputs.pred_boxes
probs = logits.softmax(-1)[0, :, :-1]  # Exclude the last class (background class)
scores, labels = probs.max(-1)

# Print detected objects
for score, label, box in zip(scores, labels, bboxes[0]):
    confidence = score.item()
    detected_object = model.config.id2label[label.item()]
    bbox = box.tolist()
    print(f"Detected {detected_object} with confidence {confidence:.3f} at location {bbox}")

Detected person with confidence 0.404 at location [0.8307971954345703, 0.8036783337593079, 0.33195486664772034, 0.3694767653942108]
Detected person with confidence 0.083 at location [0.6970181465148926, 0.06490880995988846, 0.03109738603234291, 0.02983216755092144]
Detected person with confidence 0.304 at location [0.9128207564353943, 0.5910356044769287, 0.17371846735477448, 0.34472203254699707]
Detected sheep with confidence 0.325 at location [0.1471838653087616, 0.22120633721351624, 0.08002892136573792, 0.263235479593277]
Detected person with confidence 0.479 at location [0.7009151577949524, 0.7496788501739502, 0.5923473834991455, 0.4815785884857178]
Detected person with confidence 0.268 at location [0.4219784438610077, 0.581720232963562, 0.815254271030426, 0.3237212300300598]
Detected backpack with confidence 0.084 at location [0.443172812461853, 0.657429575920105, 0.09114807844161987, 0.097919762134552]
Detected sheep with confidence 0.238 at location [0.3167034387588501, 0.2651730

In [8]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection
from PIL import Image
import torch
import os
from collections import defaultdict

# Load the image processor and model
image_processor = AutoImageProcessor.from_pretrained("microsoft/conditional-detr-resnet-50")
model = AutoModelForObjectDetection.from_pretrained("microsoft/conditional-detr-resnet-50")

# Define the base folder and categories
base_folder = r'C:\Users\rohit\Desktop\hate\hateful_memes\img\validation'
categories = ["hateful", "not_hateful"]

# Initialize results and frequency dictionaries
output_results_hateful_memes = []
output_results_not_hateful_memes = []
object_frequency_hateful_memes = defaultdict(int)
object_frequency_not_hateful_memes = defaultdict(int)

def process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = image_processor(images=[image], return_tensors="pt")
    outputs = model(**inputs)
    
    logits = outputs.logits
    bboxes = outputs.pred_boxes
    probs = logits.softmax(-1)[0, :, :-1]  # Exclude the last class (background class)
    scores, labels = probs.max(-1)
    
    results = []
    for score, label, box in zip(scores, labels, bboxes[0]):
        confidence = score.item()
        detected_object = model.config.id2label[label.item()]
        bbox = box.tolist()
        results.append({
            "object": detected_object,
            "confidence": confidence,
            "bbox": bbox
        })
    return results

# Process all images in each category
for category in categories:
    folder_path = os.path.join(base_folder, category)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        image_results = process_image(image_path)
        
        if category == "hateful":
            output_results_hateful_memes.append({"image": image_name, "results": image_results})
            for result in image_results:
                object_frequency_hateful_memes[result["object"]] += 1
        else:
            output_results_not_hateful_memes.append({"image": image_name, "results": image_results})
            for result in image_results:
                object_frequency_not_hateful_memes[result["object"]] += 1

# Save the results to text files
def save_results_to_txt(filename, results):
    with open(filename, 'w') as f:
        for entry in results:
            f.write(f"Image: {entry['image']}\n")
            for result in entry['results']:
                f.write(f"  Detected {result['object']} with confidence {result['confidence']:.3f} at location {result['bbox']}\n")
            f.write("\n")

save_results_to_txt('output_results_hateful_memes.txt', output_results_hateful_memes)
save_results_to_txt('output_results_not_hateful_memes.txt', output_results_not_hateful_memes)

# Save the object frequencies to text files
def save_frequencies_to_txt(filename, frequencies):
    with open(filename, 'w') as f:
        for object_name, count in frequencies.items():
            f.write(f"{object_name}: {count}\n")

save_frequencies_to_txt('object_frequency_hateful_memes.txt', object_frequency_hateful_memes)
save_frequencies_to_txt('object_frequency_not_hateful_memes.txt', object_frequency_not_hateful_memes)

print("Processing complete. Results saved to text files.")

Processing complete. Results saved to text files.
